In [1]:
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Sequential
from keras import layers
from keras import callbacks
from tensorflow.examples.tutorials.mnist import input_data
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training_verbosity = 2
# Download data if needed and import.
mnist = input_data.read_data_sets('tempData', one_hot=True, reshape=False)
val_images = mnist.validation.images
val_labels = mnist.validation.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use tf.data to implement this functionality.


...

Instructions for updating:
Please use tf.data to implement this functionality.


Extracting tempData\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.


Instructions for updating:
Please use tf.one_hot on tensors.


Extracting tempData\t10k-images-idx3-ubyte.gz
Extracting tempData\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Create LeNet model
model = Sequential()
model.add(Conv2D(6,
                 [5, 5],
                 input_shape=[28, 28, 1],
                 activation='relu',
                 name='conv_1'))
model.add(MaxPool2D())
model.add(Conv2D(16, [5, 5], activation='relu', name='conv_2'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(120, activation='relu', name='dense_1'))
model.add(Dense(84, activation='relu', name='dense_2'))
model.add(Dense(10, activation='softmax', name='dense_3'))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         min_delta=0,
                                         patience=10,
                                         verbose=training_verbosity,
                                         mode='auto')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.1,
                                        patience=5,
                                        verbose=training_verbosity,
                                        mode='auto',
                                        epsilon=0.0001,
                                        cooldown=0,
                                        min_lr=0)


C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train LeNet on MNIST
results = model.fit(mnist.train.images,
                    mnist.train.labels,
                    epochs=5,
                    batch_size=128,
                    verbose=2,
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stopping, reduce_lr])

loss,accuracy_original = model.evaluate(val_images, val_labels, batch_size=128, verbose=2)

model.save("./LeNet5_test/before_pruning.h5")
print('original model accuracy:', accuracy_original, '\n')

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 55000 samples, validate on 5000 samples
Epoch 1/5
 - 12s - loss: 0.3307 - acc: 0.9012 - val_loss: 0.1229 - val_acc: 0.9640
Epoch 2/5
 - 13s - loss: 0.0934 - acc: 0.9709 - val_loss: 0.0748 - val_acc: 0.9796
Epoch 3/5
 - 16s - loss: 0.0671 - acc: 0.9790 - val_loss: 0.0563 - val_acc: 0.9838
Epoch 4/5
 - 18s - loss: 0.0536 - acc: 0.9832 - val_loss: 0.0593 - val_acc: 0.9844
Epoch 5/5
 - 18s - loss: 0.0433 - acc: 0.9866 - val_loss: 0.0599 - val_acc: 0.9840
original model accuracy: 0.984 



In [6]:
layer_name = 'conv_2'

accuracy = accuracy_original

while accuracy > (0.98):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_test/after_pruning.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_test/after_pruning_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')
        

Deleting 3/16 channels from layer: conv_2
5000/5000 [==============================] - 1s 236us/step
model accuracy after pruning:  0.932 

Epoch 1/5
55000/55000 [==============================] - 18s 336us/step - loss: 0.0497 - acc: 0.9840
Epoch 2/5
  384/55000 [..............................] - ETA: 19s - loss: 0.0375 - acc: 0.9896

C:\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Anaconda\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


55000/55000 [==============================] - 18s 324us/step - loss: 0.0390 - acc: 0.9877
Epoch 3/5
55000/55000 [==============================] - 17s 307us/step - loss: 0.0335 - acc: 0.9894
Epoch 4/5
55000/55000 [==============================] - 14s 255us/step - loss: 0.0281 - acc: 0.9908
Epoch 5/5
5000/5000 [==============================] - 0s 98us/step
model accuracy after retraining:  0.9888 

Deleting 4/13 channels from layer: conv_2
5000/5000 [==============================] - 1s 149us/step
model accuracy after pruning:  0.8672 

Epoch 1/5
55000/55000 [==============================] - 17s 312us/step - loss: 0.0401 - acc: 0.9864
Epoch 2/5
55000/55000 [==============================] - 19s 350us/step - loss: 0.0287 - acc: 0.9905
Epoch 3/5
55000/55000 [==============================] - 18s 333us/step - loss: 0.0252 - acc: 0.9915
Epoch 4/5
55000/55000 [==============================] - 19s 337us/step - loss: 0.0205 - acc: 0.9931
Epoch 5/5
5000/5000 [==============================

55000/55000 [==============================] - 21s 375us/step - loss: 0.2771 - acc: 0.9101
Epoch 2/5
55000/55000 [==============================] - 18s 328us/step - loss: 0.1757 - acc: 0.9423
Epoch 3/5
55000/55000 [==============================] - 18s 324us/step - loss: 0.1495 - acc: 0.9507
Epoch 4/5
55000/55000 [==============================] - 20s 367us/step - loss: 0.1347 - acc: 0.9561
Epoch 5/5
5000/5000 [==============================] - 1s 212us/step
model accuracy after retraining:  0.9632 



In [7]:
layer_name = 'dense_1'

while accuracy > (0.97):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_test/after_pruning_dense1.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_test/after_pruning_dense1_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

In [8]:
loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128,
                          verbose=2)
print('model accuracy after retraining: ', accuracy, '\n')

model accuracy after retraining:  0.9632 



In [9]:
layer_name = 'dense_2'

while accuracy > (0.96):
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, val_images)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)

    model.save("./LeNet5_test/after_pruning_dense2.h5")
    print('model accuracy after pruning: ', accuracy, '\n')


    results = model.fit(mnist.train.images,
                        mnist.train.labels,
                        epochs=5,
                        batch_size=128,
                        callbacks=[early_stopping, reduce_lr])

    loss,accuracy = model.evaluate(val_images,
                          val_labels,
                          batch_size=128)
    model.save("./LeNet5_test/after_pruning_dense2_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

Deleting 15/84 channels from layer: dense_2
5000/5000 [==============================] - 2s 415us/step
model accuracy after pruning:  0.9594 

Epoch 1/5
55000/55000 [==============================] - 19s 345us/step - loss: 0.1213 - acc: 0.9601
Epoch 2/5
  768/55000 [..............................] - ETA: 10s - loss: 0.0797 - acc: 0.9727

C:\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Anaconda\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


55000/55000 [==============================] - 10s 190us/step - loss: 0.1121 - acc: 0.9636
Epoch 3/5
55000/55000 [==============================] - 10s 190us/step - loss: 0.1053 - acc: 0.9649
Epoch 4/5
55000/55000 [==============================] - 16s 297us/step - loss: 0.1001 - acc: 0.9670
Epoch 5/5
5000/5000 [==============================] - 1s 201us/step
model accuracy after retraining:  0.9676 

Deleting 10/69 channels from layer: dense_2
5000/5000 [==============================] - 2s 457us/step
model accuracy after pruning:  0.9588 

Epoch 1/5
55000/55000 [==============================] - 18s 323us/step - loss: 0.0963 - acc: 0.9685
Epoch 2/5
55000/55000 [==============================] - 19s 348us/step - loss: 0.0938 - acc: 0.9689
Epoch 3/5
55000/55000 [==============================] - 20s 366us/step - loss: 0.0881 - acc: 0.9707
Epoch 4/5
55000/55000 [==============================] - 18s 326us/step - loss: 0.0850 - acc: 0.9727
Epoch 5/5
5000/5000 [===========================

55000/55000 [==============================] - 20s 369us/step - loss: 0.0628 - acc: 0.9791
Epoch 2/5
55000/55000 [==============================] - 16s 295us/step - loss: 0.0583 - acc: 0.9806
Epoch 3/5
55000/55000 [==============================] - 17s 302us/step - loss: 0.0565 - acc: 0.9813
Epoch 4/5
55000/55000 [==============================] - 16s 297us/step - loss: 0.0551 - acc: 0.9813
Epoch 5/5
5000/5000 [==============================] - 1s 162us/step
model accuracy after retraining:  0.9718 

Deleting 1/19 channels from layer: dense_2
5000/5000 [==============================] - 2s 466us/step
model accuracy after pruning:  0.967 

Epoch 1/5
55000/55000 [==============================] - 21s 386us/step - loss: 0.0583 - acc: 0.9807
Epoch 2/5
55000/55000 [==============================] - 17s 305us/step - loss: 0.0556 - acc: 0.9814
Epoch 3/5
55000/55000 [==============================] - 19s 340us/step - loss: 0.0561 - acc: 0.9814
Epoch 4/5
55000/55000 [===========================

KeyboardInterrupt: 